In [ ]:
# File: 4186993
# Date: 10/27/2019
# Description: This is a Random Forest Model that predicts the probability of a batted ball being a hit 
#              based on many different batted ball characteristics. This model includes a roadmap of the 
#              entire modeling building process including, data wrangling, data cleaning, model selection, 
#              variable selection, and finally model implementation.
# Usage: File required 'Pitch_Data_v3.csv'
# References: Ashish Kumar's Learning predictive analytics with Python 2016

In [ ]:
import pandas as pd
from scipy import stats
from scipy.stats import zscore
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTENC
from sklearn.feature_selection import RFECV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Reading the pitch data csv file into a pandas data frame
data = pd.read_csv('/Users/Michael/Desktop/Pitch_Data_v3.csv')

In [ ]:
# Preview of the data frame to make sure the data was read correctly
data.head()

In [ ]:
# Creating a new column called Hit which states whether the batted ball was a hit or out
# Fielding errors are considered hits in this model
# Hit = 1, Out = 0

data['Hit'] = data['h1b'] + data['h2b'] + data['h3b'] + data['hr'] + data['fieldingError']

In [ ]:
# DATA VISUALIZATIONS
# 0 = Out, 1 = Hit 

# This is a bar chart showing the Pitch Type on the X-axis and frequency of Hits or Outs on the Y-axis
pd.crosstab(data.PitchType,data.Hit).plot(kind='bar')
plt.title('Pitch Type VS Hits/Outs')
plt.xlabel('Pitch Types')
plt.ylabel('Frequency of Hits/Outs')

# This is a bar chart showing the Batter Hitting on the X-axis VS frequency of Hits or Outs on the Y-axis
pd.crosstab(data.BatterHitting,data.Hit).plot(kind='bar')
plt.title('Batter Hitting VS Hits/Outs')
plt.xlabel('Batter Hitting')
plt.ylabel('Frequency of Hits/Outs')

# This is a bar chart showing the number of Strikes on the X-axis and frequency of Hits or Outs on the Y-axis
pd.crosstab(data.Strikes,data.Hit).plot(kind='bar')
plt.title('Strikes VS Hits/Outs')
plt.xlabel('Number of Strikes')
plt.ylabel('Frequency of Hits/Outs')

# This is a bar chart showing the number of Balls on the X-axis and frequency of Hits or Outs on the Y-axis
pd.crosstab(data.Balls,data.Hit).plot(kind='bar')
plt.title('Balls VS Hits/Outs')
plt.xlabel('Number of Balls')
plt.ylabel('Frequency of Hits/Outs')

# This is a bar chart showing the number of Outs on the X-axis and frequency of Hits or Outs on the Y-axis
pd.crosstab(data.Outs,data.Hit).plot(kind='bar')
plt.title('Outs VS Hits/Outs')
plt.xlabel('Number of Outs')
plt.ylabel('Frequency of Hits/Outs')

# This is a bar chart showing whether there is a runner on base on the X-axis and frequency of Hits or Outs on the Y-axis
pd.crosstab(data.Runners_on,data.Hit).plot(kind='bar')
plt.title('Runners On VS Hits/Outs')
plt.xlabel('Runners On')
plt.ylabel('Frequency of Hits/Outs')

# This is a scatterplot showing the relationship between Exit Speed and Angle
# Scatterplot is color based if it was a hit or out
# Purple = Out, Yellow = Hit
area = np.pi*3
plt.scatter(data['Angle'], data['ExitSpeed'], s=area, c=data['Hit'], alpha=0.5)
plt.title('ExitSpeed Vs Angle')
plt.xlabel('Angle')
plt.ylabel('Exitspeed')
plt.show()

In [ ]:
# This is a histogram showing the distribution shape of Pitch Speed in MPH
data.StartSpeed.hist()
plt.title('Histogram of Pitch Speed (MPH)')
plt.xlabel('Pitch Speed')
plt.ylabel('Frequency')

In [ ]:
# This is a histogram showing the distribution shape of Exit Speed in MPH
data.ExitSpeed.hist()
plt.title('Histogram of Exit Speed (MPH)')
plt.xlabel('Exit Speed')
plt.ylabel('Frequency')

In [ ]:
# Analyzing target variable
# Counts and shows how many hits and outs are in the data frame
# 0 = Out, 1 = Hit

data['Hit'].value_counts()

In [ ]:
# Calculates the percent of Outs in the "Hit" column 
# Calculates the percent of Hits in the "Hit" column
# Shows the imbalance in the target variable

hit = len(data[data['Hit']==1])
no_hit = len(data[data['Hit']==0])
pct_no_hit = no_hit/(no_hit+hit)
pct_hit = hit/(no_hit+hit)
print("Percentage of hits", pct_hit*100)
print("Percentage of outs", pct_no_hit*100)

In [ ]:
# Bar chart showing the unequal count of Outs vs Hits in the data frame

data['Hit'].value_counts().plot(kind='bar',figsize=(10,8),title="Number of hits",)
plt.show()

In [ ]:
# Using the function get_dummies to create 'dummy' variables based on the categorical features of 
# PitchType column in the dataset. This creates a new column for each type of pitch 

dummy_ranks = pd.get_dummies(data['PitchType'], prefix='pitch')

In [ ]:
# Joining the dummy variables for PitchType back onto our original dataset

data = data.join(dummy_ranks)

In [ ]:
# Dropping the columns that are not relevant to a batted ball being a hit or an out
# Columns were dropped based on my own person playing experience and knowledge
# Columns calledStrike, swing, swingStrike, foul, hbp, and inPlay were dropped because every ball in the dataset was put into play making those columns useless information
# Columns h1b, h2b, h3b, hr, and fieldingError were dropped because those were combined into the 'Hit' column because we only care if it was a hit or out, not what type of hit
# inPlayOut was dropped and replaced by new 'Hit' column
# PitchType was dropped because we created new dummy variable columns in replace for this column
# PinchHit was dropped because there are 0 pinch hits in this dataset

data = data.drop(columns=['GameId', 'BallparkId','GameEventId','GameTime','Season','AtBatId','TrackManUid','BatterId',
                         'BatterPositionId','PitcherId','CatcherId','HomeTeamId','AwayTeamId','Inning','InningTop',
                         'calledStrike','swing','swingStrike','foul','inPlay','h1b','h2b','h3b','hr','hbp',
                         'fieldingError','BatterHeight','PitcherHeight', 'inPlayOut', 'PitchType','PinchHit'])

In [ ]:
# Finding out if there are any null values in the data frame

data.isnull().sum()

In [ ]:
# Removing the null values from the data frame
# Decided to remove the null value from the data frame because we have such a large number of observations still without the null values

data = data.dropna()

In [ ]:
# Changing the data type of BatterHitting and PitcherThrowing to a category for easier manipulation

data["BatterHitting"] = data["BatterHitting"].astype('category')
data["PitcherThrowing"] = data["PitcherThrowing"].astype('category')

In [ ]:
# Assigning a numeric value to the corresponding category for both pitchers and hitters
# L = 0, R = 1
# After assigning 0 or 1 based on pitching or hitting side it puts it back into the original data columns

data["BatterHitting"] = data["BatterHitting"].cat.codes
data["PitcherThrowing"] = data["PitcherThrowing"].cat.codes

In [ ]:
# Calculates the Z-scores for each of the non-categorical columns in the data frame in order to spot outliers
# Z-score calculates how many standard deviations that number is away from the mean
# Puts the calculated Z-scores into new columns in the data frame

data['batterTimesFaced_Zscore'] = np.abs(stats.zscore(data['batterTimesFaced'], axis=0))
data['cumulativeBattersFaced_Zscore'] = np.abs(stats.zscore(data['cumulativeBattersFaced'], axis=0))
data['PlateAppearance_Zscore'] = np.abs(stats.zscore(data['PlateAppearance'], axis=0))
data['PitchOfPA_Zscore'] = np.abs(stats.zscore(data['PitchOfPA'], axis=0))
data['VertBreak_Zscore'] = np.abs(stats.zscore(data['VertBreak'], axis=0))
data['StartSpeed_Zscore'] = np.abs(stats.zscore(data['StartSpeed'], axis=0))
data['Px_Zscore'] = np.abs(stats.zscore(data['Px'], axis=0))
data['Pz_Zscore'] = np.abs(stats.zscore(data['Pz'], axis=0))
data['RelHeight_Zscore'] = np.abs(stats.zscore(data['RelHeight'], axis=0))
data['RelSide_Zscore'] = np.abs(stats.zscore(data['RelSide'], axis=0))
data['Extension_Zscore'] = np.abs(stats.zscore(data['Extension'], axis=0))
data['VertRelAngle_Zscore'] = np.abs(stats.zscore(data['VertRelAngle'], axis=0))
data['HorzRelAngle_Zscore'] = np.abs(stats.zscore(data['HorzRelAngle'], axis=0))
data['SpinRate_Zscore'] = np.abs(stats.zscore(data['SpinRate'], axis=0))
data['SpinAxis_Zscore'] = np.abs(stats.zscore(data['SpinAxis'], axis=0))
data['ExitSpeed_Zscore'] = np.abs(stats.zscore(data['ExitSpeed'], axis=0))
data['Angle_Zscore'] = np.abs(stats.zscore(data['Angle'], axis=0))
data['Bearing_Zscore'] = np.abs(stats.zscore(data['Bearing'], axis=0))
data['Direction_Zscore'] = np.abs(stats.zscore(data['Direction'], axis=0))

In [ ]:
# Drops the rows where outliers are present
# Outlier is defined as being greater than or equal to 3 standard deviations from the mean

data.drop(data.index[data['batterTimesFaced_Zscore'] >= 3], inplace = True)
data.drop(data.index[data['cumulativeBattersFaced_Zscore'] >= 3], inplace = True)
data.drop(data.index[data['PlateAppearance_Zscore'] >= 3], inplace = True)
data.drop(data.index[data['PitchOfPA_Zscore'] >= 3], inplace = True)
data.drop(data.index[data['VertBreak_Zscore'] >= 3], inplace = True)
data.drop(data.index[data['StartSpeed_Zscore'] >= 3], inplace = True)
data.drop(data.index[data['Px_Zscore'] >= 3], inplace = True)
data.drop(data.index[data['Pz_Zscore'] >= 3], inplace = True)
data.drop(data.index[data['RelHeight_Zscore'] >= 3], inplace = True)
data.drop(data.index[data['RelSide_Zscore'] >= 3], inplace = True)
data.drop(data.index[data['Extension_Zscore'] >= 3], inplace = True)
data.drop(data.index[data['VertRelAngle_Zscore'] >= 3], inplace = True)
data.drop(data.index[data['HorzRelAngle_Zscore'] >= 3], inplace = True)
data.drop(data.index[data['SpinRate_Zscore'] >= 3], inplace = True)
data.drop(data.index[data['SpinAxis_Zscore'] >= 3], inplace = True)
data.drop(data.index[data['ExitSpeed_Zscore'] >= 3], inplace = True)
data.drop(data.index[data['Angle_Zscore'] >= 3], inplace = True)
data.drop(data.index[data['Bearing_Zscore'] >= 3], inplace = True)
data.drop(data.index[data['Direction_Zscore'] >= 3], inplace = True)

In [ ]:
# Removes the Z-score columns from the data frame now that the outliers are found and removed

data = data.drop(columns=['batterTimesFaced_Zscore','cumulativeBattersFaced_Zscore','PlateAppearance_Zscore',
                          'PitchOfPA_Zscore','VertBreak_Zscore','StartSpeed_Zscore','Px_Zscore','Pz_Zscore',
                         'RelHeight_Zscore','RelSide_Zscore','Extension_Zscore','VertRelAngle_Zscore',
                         'HorzRelAngle_Zscore','SpinRate_Zscore','SpinAxis_Zscore','ExitSpeed_Zscore','Angle_Zscore',
                         'Bearing_Zscore','Direction_Zscore'])

In [ ]:
# X = Creates a new data frame (X) from original data frame with all columns besides 'Hit' column
# y = Creates a new data frame (y) from original data frame with ONLY the 'Hit' column, dropping all other columns

X = data.loc[:, data.columns != 'Hit']
y = data.loc[:, data.columns == 'Hit']

In [ ]:
"""
Because the data frame is imbalanced with much more Outs (0) than Hits (1) I implemented SMOTENC to over-sample and 
balance the data. SMOTENC or Synthetic Minority Over-sampling Technique for Nominal and Continuous works by finding 
two near neighbors in a minority class, producing a new point midway between the two existing points and adding that 
new point into the sample. This will make the proportion of Hits equal to the proportion of Outs.
"""
# Columns 0,1,2,3,4,5,6,7,8,9,10,27,28,29,30,31,32,33,34 are the categorical features which SMOTENC allows us to use
# After the SMOTENC test is fit to the data it is put in new variables called sample_data_X and sample_data_y
# sample_data_X contains the new balanced data with every column besides 'Hit'
# sample_data_y contains the new balanced data with only the 'Hit' column

smote = SMOTENC(categorical_features=[0,1,2,3,4,5,6,7,8,9,26,27,28,29,30,31,32,33], random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
columns = X.columns
sample_data_X,sample_data_y=smote.fit_sample(X_train, y_train)
sample_data_X = pd.DataFrame(data=sample_data_X,columns=columns )
sample_data_y= pd.DataFrame(data=sample_data_y,columns=['Hit'])

In [ ]:
# Testing to see if the SMOTENC function worked at balancing the proportion of Hits and Outs
# Prints the length of the new sampled balanced dataset
# Shows that the number of Outs and the number of Hits are equal
# Shows that the proportion of Outs and Hits are equal

print("Length of new sampled data is ", len(sample_data_X))
print("Number of outs in new sampled data", len(sample_data_y[sample_data_y['Hit']==0]))
print("Number of hits in new sampled data",len(sample_data_y[sample_data_y['Hit']==1]))
print("Proportion of outs data in new sampled data is ", len(sample_data_y[sample_data_y['Hit']==0])/len(sample_data_X))
print("Proportion of hits data in new sampled data is ",len(sample_data_y[sample_data_y['Hit']==1])/len(sample_data_X))

In [ ]:
# Checking to see if we have unique values in the dummie columns we created for PitchType
# 'pitch_KN' is only showing one unique value of 0, meaning that pitch was never thrown in our new sampled dataset

print(sample_data_X['pitch_CB'].unique())
print(sample_data_X['pitch_CH'].unique())
print(sample_data_X['pitch_CT'].unique())
print(sample_data_X['pitch_FB'].unique())
print(sample_data_X['pitch_KN'].unique())
print(sample_data_X['pitch_SI'].unique())
print(sample_data_X['pitch_SL'].unique())
print(sample_data_X['pitch_SP'].unique())

In [ ]:
# Dropping the 'pitch_KN' column because it was never thrown in our balanced sample dataset

sample_data_X = sample_data_X.drop(columns='pitch_KN')

In [ ]:
"""
RFE or Recursive feature elimination is a feature selection method that fits the model and removes the 
weakest features until the specified number of features is reached according to the f1 weighted score
RFE will tell me how many and what features are the most important for classifying the 'Hit' column
"""

#rfc = RandomForestClassifier(random_state=42)
#rfecv = RFECV(rfc, cv=3, scoring='f1_weighted')
#rfecv.fit(sample_data_X, sample_data_y)       

In [ ]:
# Shows each step in the feature elimination method and what the corresponding f1-weighted score is
#print('Grid-Scores:','\n',(rfecv.grid_scores_),'\n')

# Tells you the number of features the model should include to achieve the highest f1-weighted score
#print('Number of viable features:',(rfecv.n_features_),'\n')

# Prints out a list of numbers that shows what variables should or shouldn't be in the model
# 1 = strong fitting variable (KEEP), >1 = weak fitting variable (REMOVE)
#print('Feature Ranking:',(rfecv.ranking_))

In [ ]:
# Creates a line plot showing the relationship between Grid-Score and Number of Features Selected
# Shows how increasing the number of features will increase the Grid-Score of the model
# Graph also verifies the optimal number of features to indeed be at 9 features

#plt.figure()
#plt.xlabel("Number of Features Selected")
#plt.ylabel("Accuracy")
#plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_)
#plt.show()

In [ ]:
# FRE test concluded weakest features in the model to be all besides the columns in the 'cols' variable
# Creating a new columns variable that include only the 9 most important columns according to the FRE test
# Putting sample data with new columns in variable 'X'
# Putting sample data with only 'Hit' column in variable 'y'

cols = ['HorzBreak', 'Px','Pz','HorzRelAngle', 'SpinRate','ExitSpeed', 'Angle','Bearing','Direction' ]
X=sample_data_X[cols]
y=sample_data_y['Hit']

In [ ]:
# Performing a Random Forest Classification Model on the now treated and clean dataset
# Splitting the dataset up into a train and test dataset in order to train the model with train dataset and test how well the model is with the test dataset 
# 70% of the data is in the training set and 30% in testing set
# random_state=42 allows for the randomness to be equal throughout the model

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
rfc = RandomForestClassifier(random_state=42)
rfc.fit(X_train, y_train)

In [ ]:
# Generates the accuracy score the model had at classifying 'Hit' in the test dataset

rfc_predict = rfc.predict(X_test)
print('Random Forest Accuracy Score: {}'.format(rfc.score(X_test, y_test)))

In [ ]:
# Generates a confusion matrix which states how many hits/outs the model correctly classified
# Confusion matrix is read like the following:
#  True Negative|False Negative
# False Positive|True Positive
labels = [0,1]
confusion_matrix = confusion_matrix(y_test, rfc_predict,labels)
print(confusion_matrix)

In [ ]:
# Generates a summary of how well the model did at classifying hit or out
# Model can classify if the pitch was a hit or out with 82% accuracy
# 0 = Out, 1 = Hit

print(classification_report(y_test, rfc_predict))